In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [2]:
import pandas as pd

In [3]:
import os
import tqdm

In [4]:
batch_size = 32
device = 'cuda:0'

# Conv + LSTM 

In [94]:
class ConvLSTM(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers,device='cpu'):
        super(ConvLSTM, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size = 3, stride = 1) # Add a 1D CNN layer
        self.pool = nn.MaxPool1d(kernel_size=3,stride = 1)
        self.lstm = nn.LSTM(
            input_size=seq_len-4,
            hidden_size=n_hidden,
            num_layers=n_layers
        )
        self.linear1 = nn.Linear(in_features=n_hidden,out_features=n_hidden)
        self.linear2 = nn.Linear(in_features=n_hidden,out_features=6)
        
        
    def forward(self, sequences):
        sequences = F.relu(self.c1(sequences.view(len(sequences), 1, -1)))
        sequences = self.pool(sequences)
#         self.hidden = self.reset_hidden_state()
        
#         self.hidden = (
#         torch.zeros(self.n_layers, self.seq_len-4, self.n_hidden,device = sequences.device),
#         torch.zeros(self.n_layers, self.seq_len-4, self.n_hidden, device = sequences.device)
#         )
    
        lstm_out, _ = self.lstm(
            sequences.view(len(sequences), self.seq_len-4)
        )
        last_time_step = lstm_out.view(lstm_out.shape[0], -1, self.n_hidden)
        y_pred = F.relu(self.linear1(last_time_step))
        y_pred = F.relu(self.linear2(y_pred))
       
        return y_pred.view(-1,6)

# Read Data

In [6]:
import pickle
import numpy as np

In [7]:
virus_data_test = []
for idx, file in enumerate(os.listdir('data/test/')):
    array = pd.read_pickle(os.path.join('data/test',file))
    for i in array:
        virus_data_test.append(np.array([idx,np.array(i)]))
    del array


/tmp/ipykernel_1754392/1213216311.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  virus_data_test.append(np.array([idx,np.array(i)]))
/tmp/ipykernel_1754392/1213216311.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  virus_data_test.append(np.array([idx,np.array(i)]))
/tmp/ipykernel_1754392/1213216311.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' w

In [8]:
virus_data_val = []
for idx, file in enumerate(os.listdir('data/validation/')):
    array = pd.read_pickle(os.path.join('data/validation',file))
    for i in array:
        virus_data_val.append(np.array([idx,np.array(i)]))
    del array


/tmp/ipykernel_1754392/2924838528.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  virus_data_val.append(np.array([idx,np.array(i)]))
/tmp/ipykernel_1754392/2924838528.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  virus_data_val.append(np.array([idx,np.array(i)]))
/tmp/ipykernel_1754392/2924838528.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' whe

In [9]:
#train data
virus_data = []
for idx, file in enumerate(os.listdir('data/training/')):
    array = pd.read_pickle(os.path.join('data/training',file))
    for i in array:
        virus_data.append(np.array([idx,np.array(i)]))
    del array

#val data


# encodings_ = []
# labels_ = []
# for label, fasta_filename in enumerate(os.listdir('data/training/')):
#     with open(os.path.join('data/training',fasta_filename), 'rb') as pkfile:
#         file_encodings = pickle.load(pkfile)
#         encodings_.extend([np.array(enc) for enc in file_encodings])  # Convert nested lists to NumPy arrays
#         labels_.extend([label] * len(file_encodings))
#         del file_encodings

# encodings = np.array(encodings_, dtype=object)
# labels = np.array(labels_)
# del encodings_
# del labels_
# # Find the maximum sequence length
# max_seq_len = max([enc.shape[0] for enc in encodings])

# # Pad encodings with zeros to create tensors
# encodings_padded = []
# for enc in encodings:
#     pad_rows = max_seq_len - enc.shape[0]
#     enc_padded = np.pad(enc, (0, pad_rows), mode='constant', constant_values=0)
#     encodings_padded.append(enc_padded)
# del encodings
# encodings = np.stack(encodings_padded)
# del encodings_padded

/tmp/ipykernel_1754392/3891729634.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  virus_data.append(np.array([idx,np.array(i)]))
/tmp/ipykernel_1754392/3891729634.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  virus_data.append(np.array([idx,np.array(i)]))
/tmp/ipykernel_1754392/3891729634.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creati

In [10]:
virus_data = np.array(virus_data)
virus_data_val = np.array(virus_data_val)

In [11]:
virus_data_test= np.array(virus_data_test)

In [12]:
max_seq_len_train = max([enc.shape[0] for enc in virus_data[...,1]])
max_seq_len_val = max([enc.shape[0] for enc in virus_data_val[...,1]])
max_seq_len_test = max(enc.shape[0] for enc in virus_data_test[...,1])


In [13]:
max_seq_len = max([max_seq_len_train,max_seq_len_val,max_seq_len_test])

# Pad encodings with zeros to create tensors
encodings_padded = []
for enc in  virus_data[...,1]:
    pad_rows = max_seq_len - enc.shape[0]
    enc_padded = np.pad(enc, (0, pad_rows), mode='constant', constant_values=0)
    encodings_padded.append(enc_padded)


In [14]:
encodings_padded_val = []
for enc in  virus_data_val[...,1]:
    pad_rows = max_seq_len - enc.shape[0]
    enc_padded = np.pad(enc, (0, pad_rows), mode='constant', constant_values=0)
    encodings_padded_val.append(enc_padded)
    
encodings_padded_test = []
for enc in  virus_data_test[...,1]:
    pad_rows = max_seq_len - enc.shape[0]
    enc_padded = np.pad(enc, (0, pad_rows), mode='constant', constant_values=0)
    encodings_padded_test.append(enc_padded)

In [15]:
def normalize_encodings(encodings):
    mean = np.mean(encodings)
    std = np.std(encodings)
    return (encodings -mean)/std

encodings_padded = normalize_encodings(encodings_padded)
encodings_padded_val = normalize_encodings(encodings_padded_val)
encodings_padded_test = normalize_encodings(encodings_padded_test)

# Dataset

In [16]:
class VirusDataset(Dataset):
    def __init__(self,data,label):
        self.data = data
        self.label = label
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return torch.Tensor(self.data[idx]),self.label[idx]

In [25]:
train_dataset = VirusDataset(encodings_padded,virus_data[...,0])
val_dataset = VirusDataset(encodings_padded_val,virus_data_val[...,0])
test_dataset = VirusDataset(encodings_padded_test,virus_data_test[...,0])

In [26]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True)


# Training

In [95]:
model = ConvLSTM(n_features = 1 , n_hidden = 64, seq_len = max_seq_len, n_layers = 2,device=device)
model = nn.DataParallel(model, device_ids=[0,1,2])


In [96]:
model.to(device)

DataParallel(
  (module): ConvLSTM(
    (c1): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
    (pool): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
    (lstm): LSTM(30479, 64, num_layers=2)
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (linear2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [21]:
learning_rate = 0.00001
num_epochs = 10
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [104]:
def train_epoch(model, dataloader, criterion, optimizer, val_loader = None):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0
    for inputs, targets in tqdm.tqdm(dataloader):
        # Move tensors to the correct device
        inputs = inputs.to(device)
        targets = targets.type(torch.LongTensor).to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        # print(outputs.squeeze().shape)
#         print(outputs.dtype)
#         print(targets.dtype)
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
#         if idx % 200 == 0:
#             print(f"batch [{idx + 1}/{len(dataloader)}], outputs: {ouputs}")
        #Calculate the number of correct predictions
        predicted = torch.argmax(outputs, -1)
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)
        
    for inputs, targets in tqdm.tqdm(val_loader):
        # Move tensors to the correct device
        inputs = inputs.to(device)
        targets = targets.type(torch.LongTensor).to(device)

        # Forward pass
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), targets)

    
        val_running_loss += loss.item()
        predicted = torch.argmax(outputs, -1)

        val_correct_predictions += (predicted == targets).sum().item()
        val_total_predictions += targets.size(0) 
        
    average_loss = running_loss / len(dataloader)
    accuracy = correct_predictions / total_predictions
    val_average_loss = val_running_loss / len(val_loader)
    val_accuracy = val_correct_predictions / val_total_predictions
    return average_loss, accuracy, val_average_loss, val_accuracy

In [105]:
history = {
    'acc':[],
    'val_acc':[],
    'loss' : [],
    'val_loss': []
}
for epoch in range(num_epochs):
    train_loss, train_accuracy,val_loss,val_accuracy = train_epoch(model, train_loader, criterion, optimizer,val_loader)
    history['acc'].append(train_accuracy)
    history['val_acc'].append(val_accuracy)
    history['loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    
    print(history)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:01<00:00, 52.94it/s]


{'acc': [0.15965277777777778], 'val_acc': [0.1622222222222222], 'loss': [1.7932092356681824], 'val_loss': [1.7933156239359003]}
Epoch [1/10], Loss: 1.7932, Accuracy: 0.1597


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:01<00:00, 53.84it/s]


{'acc': [0.15965277777777778, 0.15527777777777776], 'val_acc': [0.1622222222222222, 0.15611111111111112], 'loss': [1.7932092356681824, 1.7934254471460978], 'val_loss': [1.7933156239359003, 1.7931947248023854]}
Epoch [2/10], Loss: 1.7934, Accuracy: 0.1553


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:01<00:00, 53.35it/s]


...

{'acc': [0.15965277777777778, 0.15527777777777776, 0.15833333333333333, 0.15590277777777778, 0.15916666666666668, 0.1579861111111111, 0.15368055555555554, 0.16020833333333334], 'val_acc': [0.1622222222222222, 0.15611111111111112, 0.16722222222222222, 0.15944444444444444, 0.15722222222222224, 0.16333333333333333, 0.15388888888888888, 0.15333333333333332], 'loss': [1.7932092356681824, 1.7934254471460978, 1.7932910238371955, 1.7933145252863567, 1.7932477378845215, 1.7932729069391886, 1.7933854148122998, 1.7932749263445535], 'val_loss': [1.7933156239359003, 1.7931947248023854, 1.7930588931368108, 1.7936348789616634, 1.7932150259352566, 1.7933617934846042, 1.7932643576672203, 1.793335803768091]}
Epoch [8/10], Loss: 1.7933, Accuracy: 0.1602


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:01<00:00, 53.41it/s]


{'acc': [0.15965277777777778, 0.15527777777777776, 0.15833333333333333, 0.15590277777777778, 0.15916666666666668, 0.1579861111111111, 0.15368055555555554, 0.16020833333333334, 0.15444444444444444], 'val_acc': [0.1622222222222222, 0.15611111111111112, 0.16722222222222222, 0.15944444444444444, 0.15722222222222224, 0.16333333333333333, 0.15388888888888888, 0.15333333333333332, 0.1661111111111111], 'loss': [1.7932092356681824, 1.7934254471460978, 1.7932910238371955, 1.7933145252863567, 1.7932477378845215, 1.7932729069391886, 1.7933854148122998, 1.7932749263445535, 1.7934537792205811], 'val_loss': [1.7933156239359003, 1.7931947248023854, 1.7930588931368108, 1.7936348789616634, 1.7932150259352566, 1.7933617934846042, 1.7932643576672203, 1.793335803768091, 1.7932796561927127]}
Epoch [9/10], Loss: 1.7935, Accuracy: 0.1544


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:01<00:00, 53.94it/s]

{'acc': [0.15965277777777778, 0.15527777777777776, 0.15833333333333333, 0.15590277777777778, 0.15916666666666668, 0.1579861111111111, 0.15368055555555554, 0.16020833333333334, 0.15444444444444444, 0.15625], 'val_acc': [0.1622222222222222, 0.15611111111111112, 0.16722222222222222, 0.15944444444444444, 0.15722222222222224, 0.16333333333333333, 0.15388888888888888, 0.15333333333333332, 0.1661111111111111, 0.16666666666666666], 'loss': [1.7932092356681824, 1.7934254471460978, 1.7932910238371955, 1.7933145252863567, 1.7932477378845215, 1.7932729069391886, 1.7933854148122998, 1.7932749263445535, 1.7934537792205811, 1.793329763677385], 'val_loss': [1.7933156239359003, 1.7931947248023854, 1.7930588931368108, 1.7936348789616634, 1.7932150259352566, 1.7933617934846042, 1.7932643576672203, 1.793335803768091, 1.7932796561927127, 1.793527647068626]}
Epoch [10/10], Loss: 1.7933, Accuracy: 0.1562


In [ ]:
def evaluate(model, dataloader):
    model.train()
    correct_predictions = 0
    total_predictions = 0
    for inputs, targets in tqdm.tqdm(dataloader):
        # Move tensors to the correct device
        idx += 1
        inputs = inputs.to(device)
        targets = targets.type(torch.LongTensor).to(device)

        # Forward pass
        outputs = model(inputs)


        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    accuracy = correct_predictions / total_predictions
    
    return average_loss, accuracy